In [ ]:
import zipfile
import os

# 파일 경로 설정
zip_file_path = r'../wafer_anal_prj/LSWMD.pkl.zip'
extract_dir = r'../wafer_anal_prj/extracted_files'  # 압축을 풀 위치 설정

# zip 파일 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"파일이 {extract_dir}에 압축 풀렸습니다.")


In [ ]:
import os
import pickle

import sys
import pandas as pd
sys.modules['pandas.indexes'] = pd.core.indexes

# 실제 디렉토리 경로 설정
extract_dir = '../wafer_anal_prj/extracted_files'  # 실제 경로로 변경

# pkl 파일 경로 설정
pkl_file_path = os.path.join(extract_dir, 'LSWMD.pkl')

# 파일 경로 확인
if not os.path.exists(pkl_file_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pkl_file_path}")

# pkl 파일 로드
with open(pkl_file_path, 'rb') as file:
    data = pickle.load(file, encoding="latin1")  # encoding을 latin1로 변경

print("데이터가 성공적으로 로드되었습니다.")
print(data.head())  # 데이터의 처음 몇 줄 확인

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

# pkl 파일 경로 설정
extract_dir = 'D:/wafer_anal_prj/extracted_files'  # 실제 경로로 변경
pkl_file_path = os.path.join(extract_dir, 'LSWMD.pkl')

# 파일 경로 확인
if not os.path.exists(pkl_file_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pkl_file_path}")

# pkl 파일 로드
with open(pkl_file_path, 'rb') as file:
    data = pickle.load(file, encoding="latin1")  # encoding을 latin1로 변경

# 데이터가 DataFrame인지 확인
if isinstance(data, pd.DataFrame):
    df_withpattern = data
else:
    # 데이터가 dict 또는 list 형태라면 DataFrame으로 변환
    df_withpattern = pd.DataFrame(data)

# 데이터 구조 확인
print(df_withpattern.info())
print(df_withpattern.head())

# 결측값 처리
df_withpattern.dropna(subset=['waferMap', 'failureType'], inplace=True)

# failureType 인코딩
labels2 = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Random', 'Scratch', 'Near-full']

def encode_failure_type(x):
    if x and x[0][0] in labels2:
        return labels2.index(x[0][0])
    else:
        return -1  # labels2에 없는 값은 -1로 처리

df_withpattern['failureType_encoded'] = df_withpattern['failureType'].apply(encode_failure_type)

# waferMap 정규화
df_withpattern['waferMap_normalized'] = df_withpattern['waferMap'].apply(lambda x: np.array(x) / 255.0)

# 필요 없는 열 제거
df_withpattern.drop(columns=['failureType'], inplace=True)

# 전처리된 데이터 저장
df_withpattern.to_pickle('../wafer_anal_prj/processed_df_withpattern.pkl')
print("전처리된 데이터가 저장되었습니다.")

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# 전처리된 데이터 로드
processed_data_path = '../wafer_anal_prj/processed_df_withpattern.pkl'
df_withpattern = pd.read_pickle(processed_data_path)

# 데이터 분할 비율 설정
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 데이터 셔플링 및 분할
train_data, temp_data = train_test_split(df_withpattern, test_size=(1 - train_ratio), random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42)

# 분할된 데이터 크기 확인
print(f"Training data: {len(train_data)} rows")
print(f"Validation data: {len(val_data)} rows")
print(f"Test data: {len(test_data)} rows")

# 분할된 데이터 저장
train_data.to_pickle('../wafer_anal_prj/train_data.pkl')
val_data.to_pickle('../wafer_anal_prj/val_data.pkl')
test_data.to_pickle('../wafer_anal_prj/test_data.pkl')

print("Training, validation, and test data가 저장되었습니다.")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# 분류된 데이터 로드
train_data = pd.read_pickle('../wafer_anal_prj/train_data.pkl')
val_data = pd.read_pickle('../wafer_anal_prj/val_data.pkl')
test_data = pd.read_pickle('../wafer_anal_prj/test_data.pkl')

# 데이터 확인
print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

In [ ]:
from scipy.ndimage import zoom
import numpy as np

def resize_array(arr, target_shape):
    arr = np.array(arr)
    zoom_factors = (target_shape / arr.shape[0], target_shape / arr.shape[1])
    return zoom(arr, zoom_factors, order=1)  # bilinear

target_shape = 32  # 또는 64 등 더 작은 값으로 조정

train_data['waferMap_normalized'] = train_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))
val_data['waferMap_normalized'] = val_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))
test_data['waferMap_normalized'] = test_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))

X_train = np.stack(train_data['waferMap_normalized'].values).astype(np.float32)
y_train = train_data['failureType_encoded']

X_val = np.stack(val_data['waferMap_normalized'].values).astype(np.float32)
y_val = val_data['failureType_encoded']

X_test = np.stack(test_data['waferMap_normalized'].values).astype(np.float32)
y_test = test_data['failureType_encoded']

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']

# -1(분류 불가) 데이터 제거
train_data = train_data[train_data['failureType_encoded'] != -1]
val_data = val_data[val_data['failureType_encoded'] != -1]
test_data = test_data[test_data['failureType_encoded'] != -1]

# 리사이즈 적용
from scipy.ndimage import zoom
def resize_array(arr, target_shape):
    arr = np.array(arr)
    zoom_factors = (target_shape / arr.shape[0], target_shape / arr.shape[1])
    return zoom(arr, zoom_factors, order=1)

target_shape = 32
train_data['waferMap_normalized'] = train_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))
val_data['waferMap_normalized'] = val_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))
test_data['waferMap_normalized'] = test_data['waferMap_normalized'].apply(lambda x: resize_array(x, target_shape))

# X, y 생성
X_train = np.stack(train_data['waferMap_normalized'].values).astype(np.float32)
y_train = train_data['failureType_encoded']
X_val = np.stack(val_data['waferMap_normalized'].values).astype(np.float32)
y_val = val_data['failureType_encoded']
X_test = np.stack(test_data['waferMap_normalized'].values).astype(np.float32)
y_test = test_data['failureType_encoded']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 랜덤시드 40으로 설정
np.random.seed(40)

# 2D 이미지를 1D 벡터로 변환 (랜덤포레스트는 1D 입력 필요)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# 모델 학습
clf = RandomForestClassifier(n_estimators=100, random_state=40)
clf.fit(X_train_flat, y_train)

# 검증 데이터 평가
y_val_pred = clf.predict(X_val_flat)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# 테스트 데이터 평가
y_test_pred = clf.predict(X_test_flat)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
import numpy as np

def pad_to_square(arr, pad_value=0):
    arr = np.array(arr)
    h, w = arr.shape
    size = max(h, w)
    pad_h = (size - h) // 2
    pad_w = (size - w) // 2
    arr_padded = np.pad(arr, ((pad_h, size - h - pad_h), (pad_w, size - w - pad_w)), 
                        mode='constant', constant_values=pad_value)
    return arr_padded

def preprocess_wafer(arr, target_shape):
    arr = pad_to_square(arr)
    return resize_array(arr, target_shape)

# 패딩 후 리사이즈 적용
train_data['waferMap_normalized'] = train_data['waferMap_normalized'].apply(lambda x: preprocess_wafer(x, 32))
val_data['waferMap_normalized'] = val_data['waferMap_normalized'].apply(lambda x: preprocess_wafer(x, 32))
test_data['waferMap_normalized'] = test_data['waferMap_normalized'].apply(lambda x: preprocess_wafer(x, 32))